In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import Tuple


## Dataset em ingles apenas para testes iniciais de funcionamento do código
https://github.com/paul-rottger/hatecheck-data/blob/main/all_cases.csv

In [ ]:
def simple_preprocess(text: str) -> str:
    if not isinstance(text, str):
        return ''
    # basic normalizations used in the original pipeline
    text = text.replace('&amp;', ' and ')
    text = text.replace('&', ' and ')
    text = re.sub(r'[\r\n]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def load_synthetic(path: str, text_col: str = 'text', label_col: str = 'label') -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv("data/all_cases.csv") 
    
    df['text'] = df['test_case']
    df["label"] = df["label_gold"].apply(lambda x: 1 if x == "hateful" else 0)

    df = df[[text_col, label_col]].dropna().reset_index(drop=True)
    df = df.rename(columns={text_col: 'text', label_col: 'label'})
    df['text'] = df['text'].astype(str).map(simple_preprocess)
    df['label'] = df['label'].astype(int)

    df_train, df_test = train_test_split(df, test_size = 0.25, random_state=42, shuffle=True)
    df_train, df_val = train_test_split(df_train, test_size = 0.20, random_state=42, shuffle=True)
    return df_train, df_val, df_test

train_df, val_df, test_df = load_synthetic(path="data/all_cases.csv")

train_df.to_csv("data/1en_train.csv") #60%
val_df.to_csv("data/1en_val.csv") #15%
test_df.to_csv("data/1en_test.csv") #25%

## Datasets em português (splits individuais)
- OLID-BR: https://huggingface.co/datasets/dougtrajano/olid-br
- hateBR: https://github.com/franciellevargas/HateBR/blob/main/dataset/HateBR.csv

Vou fazer o split em train, val e test pra deixar pronto pros experimentos (usando um único dataset por vez)

#### 1. Olid-br

In [2]:
from datasets import load_dataset
dataset = load_dataset("dougtrajano/olid-br")

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()

df_train["label"] = df_train["is_offensive"].apply(lambda x: 1 if x == "OFF" else 0)
df_test["label"] = df_test["is_offensive"].apply(lambda x: 1 if x == "OFF" else 0)


In [3]:
# Juntar todo o dataset olidbr em um csv só

combined_df = pd.concat([df_train, df_test], ignore_index=True)
combined_df.to_csv("data/olidbr/full.csv", index=False)

In [4]:
# Splitar df_train em train (60%) e val (15%)
# Como df_train representa 75% do total, queremos que a validação seja:
# val_ratio = (15% total) / (75% total após split inicial)

val_ratio = 0.15 / 0.75   # = 0.20

df_train, df_val = train_test_split(
    df_train,
    test_size=val_ratio,
    random_state=42,
    shuffle=True
)

df_train.to_csv("data/olidbr/train.csv", index=False) #60%
df_val.to_csv("data/olidbr/val.csv", index=False) #15%
df_test.to_csv("data/olidbr/test.csv", index=False) #25%

In [7]:
assert len(combined_df)==(len(df_train)+len(df_test)+len(df_val))

#### 2. HateBR

In [18]:
hatebr = pd.read_csv("data/hatebr/full.csv")
hatebr = hatebr.rename(columns={
    "comentario": "text",
    "label_final": "label"
})
hatebr.head()

,id,text,anotator1,anotator2,anotator3,label,links_post,account_post
0,1,Mais um lixo,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
1,2,Essa nao tem vergonha na cara!!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
2,3,Essa mulher é doente.pilantra!,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
3,4,Comunista safada...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli
4,5,Vagabunda. Comunista. Mentirosa. O povo chilen...,1,1,1,1,https://www.instagram.com/p/B2uThqdH9xI/,Carla Zambelli


In [19]:
hatebr_train, hatebr_test = train_test_split(hatebr, test_size = 0.25, random_state=42, shuffle=True)
hatebr_train, hatebr_val = train_test_split(hatebr_train, test_size = 0.20, random_state=42, shuffle=True)

In [24]:
assert len(hatebr)==(len(hatebr_train)+len(hatebr_test)+len(hatebr_val))

In [25]:
hatebr_train.to_csv("data/hatebr/train.csv", index=False) #60%
hatebr_val.to_csv("data/hatebr/val.csv", index=False) #15%
hatebr_test.to_csv("data/hatebr/test.csv", index=False) #25%

## Cross-dataset
- OLID-BR: https://huggingface.co/datasets/dougtrajano/olid-br
- hateBR: https://github.com/franciellevargas/HateBR/blob/main/dataset/HateBR.csv

A ideia aqui é usar um dataset para train+val e o outro inteiro para test. 
Vou avaliar os 2 casos: 
- treinar em olidbr e testar em hatebr (vou splitar o olidbr em train e val, e usar o hatebr completo como test)
- treinar em hatebr e avaliar em olidbr (vou splitar o hatebr em train e val, e usar o olidbr completo como test)
